<a href="https://colab.research.google.com/github/saitejakarre/Saiteja/blob/main/Text_Summarization_using_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

df = load_dataset("knkarthick/dialogsum")

In [ ]:
article_1 = df['train'][1]['dialogue']

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")

In [ ]:
def preprocess_function(batch):
    source = batch['dialogue']
    target = batch["summary"]
    source_ids = tokenizer(source, truncation=True, padding="max_length", max_length=128)
    target_ids = tokenizer(target, truncation=True, padding="max_length", max_length=128)
    labels = target_ids["input_ids"]
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels]

    return {
        "input_ids": source_ids["input_ids"],
        "attention_mask": source_ids["attention_mask"],
        "labels": labels
    }

df_source = df.map(preprocess_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="/content",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    report_to='none',
    remove_unused_columns=False
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_source["train"],
    eval_dataset=df_source["test"]
)

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

In [ ]:
model.save_pretrained("/content/SaiTejaKarreTS")
tokenizer.save_pretrained("/content/SaiTejaKarreTS")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("/content/SaiTejaKarreTS")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/SaiTejaKarreTS")

def summarize(blog_post):
    inputs = tokenizer(blog_post, max_length=1024, truncation=True, return_tensors="pt")

    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

Full_Text = """
Lord Venkateshwara, also known as Srinivasa, Balaji, or Venkatachalapati, is a revered form of Lord Vishnu, primarily worshipped at the sacred Tirumala Venkateswara Temple in Tirupati, Andhra Pradesh, India. Considered the “Lord of the Seven Hills,” He is believed to have manifested in the Kali Yuga to protect humanity and grant blessings to His devotees. The deity’s idol is renowned for its unique beauty, adorned with precious jewels and ornaments, and is one of the most visited and venerated forms of Vishnu worldwide.

According to legend, Lord Venkateshwara descended to earth to save mankind from the trials and tribulations of the present age. The most popular story narrates that Vishnu took the form of Srinivasa and came to Tirumala to marry Goddess Padmavathi, the incarnation of Lakshmi. To arrange this divine wedding, He borrowed wealth from Kubera, the god of riches. Even today, devotees believe they are repaying this cosmic debt through offerings at the temple, which explains the tradition of immense donations.

The Tirumala temple is one of the richest and most visited places of worship globally, attracting millions of pilgrims every year. The daily rituals, chants, and the famous “Suprabhatam” prayers bring an atmosphere of deep devotion. The prasadam “Tirupati Laddu” has gained global recognition and holds a Geographical Indication (GI) tag.

Spiritually, Lord Venkateshwara represents hope, compassion, and divine justice. His devotees believe that surrendering at His feet removes sins, fulfills wishes, and grants peace. The sacred mantra “Om Namo Venkatesaya” is chanted with unwavering faith by millions, symbolizing surrender to the Lord’s will.

Beyond being a religious figure, Lord Venkateshwara has influenced culture, music, literature, and traditions for centuries. The annual Brahmotsavam festival draws devotees from across the world, creating a vibrant celebration of faith and unity. His temple stands as a testament to both architectural grandeur and the enduring power of devotion.

In essence, Lord Venkateshwara is not just a deity for His followers; He is a living presence, a protector, and a beacon of hope in challenging times, whose blessings continue to guide devotees toward spiritual fulfillment and worldly prosperity
"""

summary = summarize(Full_Text)
print("Summary:", summary)
